# JillyAtlanta Sales

***

In [1]:
import pandas as pd
import numpy as np
import glob, datetime
from matplotlib import pyplot as plt

***
## Etsy

Get orders, nets, and refunds.

In [2]:
fiels = glob.glob('./etsy/EtsySales/*.csv')
df1 = pd.DataFrame()
for fiel in fiels:
    dfi = pd.read_csv(fiel)
    df1 = pd.concat([df1, dfi])
df1 = df1[['Order Date', 
           'Order ID', 
           'Net Amount', 
           'Refund Amount']]
df1.columns = ['date', 'order_id', 'net_amount', 'refund_amount']
df1['date'] = pd.to_datetime(df1['date'])
df1.sort_values(by='date', inplace=True)
df1.head(20)

,date,order_id,net_amount,refund_amount
0,2014-06-16,144656004,56.01,0.0
1,2014-08-30,155967399,96.75,8.0
2,2014-08-31,156150920,44.37,6.0
3,2014-09-02,156470823,90.93,0.0
4,2014-09-26,160466268,44.37,6.0
5,2014-10-15,163749398,50.19,0.0
6,2014-11-17,169564370,50.19,0.0
7,2014-11-22,170698208,52.13,0.0
8,2014-11-28,172266166,52.13,0.0
9,2014-11-30,172879375,50.19,0.0


Get order_id sales items.

In [3]:
fiels = glob.glob('./etsy/EtsySoldOrderItems/*.csv')
df2 = pd.DataFrame()
for fiel in fiels:
    dfi = pd.read_csv(fiel)
    df2 = pd.concat([df2, dfi])
#df2.head(20)

Get df with date, order_id, and item_name

In [4]:
df2 = df2[['Sale Date',
           'Order ID', 
           'Item Name']]
df2.columns = ['date', 'order_id', 'item_name']
df2['date'] = pd.to_datetime(df2['date'])
df2.sort_values(by='date', inplace=True)
#df2.reset_index()
df2.head(20)

,date,order_id,item_name
23,2014-06-16,144656004,Garden Skirt
22,2014-08-30,155967399,"Reserved Listing: girl organic shorts, blue or..."
21,2014-08-30,155967399,Custom order--Size 12/14
20,2014-08-31,156150920,"Reserved Listing: girls shorts, organic chamb..."
19,2014-09-02,156470823,"Reserved Listing: girl top, organic cotton, ye..."
18,2014-09-02,156470823,"Reserved Listing: organic cotton twill, pink s..."
17,2014-09-26,160466268,"Reserved Listing: Girl Organic Shorts, cotton ..."
16,2014-10-15,163749398,"Girl Holiday Skirt, sequined girl skirt,knee l..."
15,2014-11-17,169564370,"Girl Holiday Skirt,sequined girl skirt,knee le..."
13,2014-11-19,170065489,"Girl Holiday Skirt,sequined girl skirt,knee le..."


Identify item_name as PDF or not.  First, make a list of all the PDF listings.

In [5]:
listings = pd.read_csv('listings_names.csv')
listings = listings[listings.PDF != 0]
listings.head()
pdfs = [x.strip("\" '") for x in listings.item_title.values]
for each in pdfs:
    if 'pdf'.casefold() in each.casefold(): pass
    else: 
        print(each)
        1/0
#len(pdfs)

Then, in df2, add a column 'is_pdf' = False (not a pdf) or True (pdf).  Once this is done, remove column with item_name.  Finally, remove duplicate columns.

In [6]:
def is_pdf(item_name):
    return item_name in pdfs

df2['pdf'] = df2['item_name'].apply(is_pdf)
df2 = df2[['order_id', 'pdf']]
df2.drop_duplicates(inplace=True)
df2.sort_values(by='order_id', inplace=True)
#df2.head()
#df2.tail()

Merge df1 and df2 to give dataframe of all sales.  Then remove sales that were not pdfs.

In [7]:
df3 = pd.merge(df1, df2, on='order_id', how='outer')
df3 = df3[df3.pdf == True]
print(len(df3.index))
df3.dropna()
print(len(df3.index))
#df3.tail()

11921
11921


In [8]:
df3 = df3.dropna(axis=0)
#df3.tail()

In [9]:
df3['year_month'] = df3['date'].dt.to_period('M')
df3['sales'] = df3['net_amount'] - df3['refund_amount']
#df3.head()

In [10]:
df4 = df3[['year_month', 'sales']]
#df4.tail()

In [11]:
df5 = df4.groupby('year_month').sum().reset_index()
#df5.head()

In [12]:
df5['year_month'] = df5.year_month.values.astype('datetime64[M]')
df5.dtypes

year_month    datetime64[ns]
sales                float64
dtype: object

In [13]:
dates = [datetime.datetime.str(x) for x in list(df5['year_month'])]
sales_etsy = list(df5['sales'])

plt.figure(figsize=(20,4))

plt.bar(dates, sales_etsy)
ax = plt.gca() # grab the current axis
#ax.set_xticks([1, 13, 25, 37, 49, 61, 73, 85]) # choose which x locations to have ticks
#ax.set_xticklabels = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'] # set the labels to display at those ticks

max_value = dates.max()
min_value = dates.min()
number_of_steps = 12
l = np.arange(min_value, max_value+1, number_of_steps)

ax.set(xticks=l, xticklabels=l)
plt.show()

AttributeError: type object 'datetime.datetime' has no attribute 'str'

In [ ]:
names = ['group_a', 'group_b', 'group_c']
values = [1, 10, 100]

plt.figure(figsize=(9, 3))

plt.subplot(131)
plt.bar(names, values)
plt.subplot(132)
plt.scatter(names, values)
plt.subplot(133)
plt.plot(names, values)
plt.suptitle('Categorical Plotting')
plt.show()

plt.clf()
plt.plot(x,y,'o-')
ax = plt.gca() # grab the current axis
ax.set_xticks([1,2,3]) # choose which x locations to have ticks
ax.set_xticklabels([1,"key point",2]) # set the labels to display at those ticks

***
## Shopify

Get Shopify sales data.

In [ ]:
df2 = pd.read_csv('./shopify/sales.csv')
#df.head()

Get rid of records of days that have no sales.

In [ ]:
df2 = df2.dropna(axis=0)
#df.head()

Garner columns of interest.

In [ ]:
df2 = df2[['day', 'net_sales']]

Rename columns to match Etsy dataframe.

In [ ]:
df2.columns = ['date', 'sales']
df2.head()

Change date column to datetime object.

In [ ]:
df2['date'] = pd.to_datetime(df2['date'])

Reformat dates so we can group and sum.

In [ ]:
df2['date'] = df2['date'].dt.to_period('M')
df2 = df2.groupby('date').sum().reset_index()
df2.head()

In [ ]:
df2['date'] = df2.date.values.astype('datetime64[M]')
#df2.dtypes
#print(df2)

In [ ]:
df_shopify = df2
dates = [str(x) for x in list(df2['date'])]
sales_shopify = list(df2['sales'])
plt.figure(figsize=(20,4))
plt.bar(dates, sales_shopify)
locs, labels = plt.xticks()
xticks = ['2021', '2022']
new_locs = []
i = 9
while i <= locs[-1] + 1:
    new_locs.append(i)
    i += 12
print(new_locs)
plt.xticks(new_locs, xticks, horizontalalignment='right')
type(dates[3])

***
## Combine Etsy and Shopify

In [ ]:
df_etsy.columns = ['date', 'sales_etsy']
df_shopify.columns = ['date', 'sales_shopify']
df_merge = pd.merge(df_etsy, df_shopify, on='date', how='outer')
df_merge = df_merge.replace(np.nan,0)
df_merge = pd.DataFrame(df_merge)
df = pd.DataFrame()
df['dates'] = [str(x) for x in list(df_merge['date'])]
df['etsy'] = df_merge['sales_etsy']
df['shopify'] = df_merge['sales_shopify']
df.head()

In [ ]:
plt.figure(figsize=(12,4))
df.plot(kind='bar', stacked=True)
plt.title('JillyAtlanta Sales')

In [ ]:
plt.bar(df.dates.values, df.etsy.values, color='r')
plt.bar(df.dates.values, df.shopify.values, bottom=df.etsy.values, color='b')
plt.figure(figsize=(12,4))
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend(['Etsy', 'Shopify'])
plt.title('JillyAtlanta Sales')
plt.show()